In [1]:
from transformers import AutoTokenizer, AutoModel
from tokenizers import Tokenizer
# tokenizer = AutoTokenizer.from_pretrained('human_gpt2-v1')
# tokenizer.pad_token = tokenizer.eos_token

# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})

new_tokenizer = Tokenizer.from_file("dna_eng_bpe_dict.json")
#或者下面方法
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast(tokenizer_object=new_tokenizer)
tokenizer.pad_token = tokenizer.eos_token

/home/maris/miniconda3/envs/dnagpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maris/miniconda3/envs/dnagpt/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
full_model = AutoModel.from_pretrained('gpt_dna_v0')
#model.config.eos_token_id
# print(model.config.pad_token_id)
#model.config.pad_token_id = model.config.eos_token_id

In [3]:
gena_module_name = full_model.__class__.__module__
print(gena_module_name)

transformers.models.gpt2.modeling_gpt2


In [4]:
import importlib
# available class names:
# - BertModel, BertForPreTraining, BertForMaskedLM, BertForNextSentencePrediction,
# - BertForSequenceClassification, BertForMultipleChoice, BertForTokenClassification,
# - BertForQuestionAnswering
# check https://huggingface.co/docs/transformers/model_doc/bert
myclass = importlib.import_module(gena_module_name)
#dir(myclass)

In [5]:
cls = getattr(importlib.import_module(gena_module_name), 'GPT2ForSequenceClassification')
cls

transformers.models.gpt2.modeling_gpt2.GPT2ForSequenceClassification

In [6]:
model = cls.from_pretrained('gpt_dna_v0', num_labels=2)
#dir(model)
#model.config.eos_token_id
#print(model.config.pad_token_id)
model.config.pad_token_id = model.config.eos_token_id
#model.resize_token_embeddings(len(tokenizer))

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt_dna_v0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from datasets import load_dataset
# load ~11k samples from promoters prediction dataset
dataset = load_dataset("yurakuratov/example_promoters_2k")['train'].train_test_split(test_size=0.1)

Using the latest cached version of the dataset since yurakuratov/example_promoters_2k couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/maris/.cache/huggingface/datasets/yurakuratov___example_promoters_2k/default/0.0.0/78939766a636f4b4b852ef3affbcb0bbb2f84e5b (last modified on Tue Aug 13 12:06:03 2024).


In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sequence', 'promoter_presence'],
        num_rows: 10656
    })
    test: Dataset({
        features: ['sequence', 'promoter_presence'],
        num_rows: 1184
    })
})

In [9]:
dataset['train'][0]

{'sequence': 'GCGCTGAAGAAAGAACACCTGAATCCGGAGCCCCAGCCCCCGTCGCTCGTTTGCGCGACCCCAAAACCTGGCGGGACGAGGGTCGCCCGATGTCTAGCTGGAGGAGTTTGGGCTTGGCGCCTCCTTCGTGTAGTGGCTGAAGCTTCCCGGGTGCTCCTCATTCCTGGGAAGGGAGTCGGGTGATGGGAACGGTGGGAGCTCAGCGCTCCGAGTCTGCGGCCGGGGGCCAGCGGGGCTGAAACGCGCCTCCTGAGTGTTGAAAGCGGCAGGCGCCGGCCTTTGCACCCAGAGGCCAGGGCGGACGGGCAGGATAAGCAGGCACTGGGCCCAAGCCGCCCTTGGAGGCTCTGCCTGTTCTGATCTGAAGGCGGAGGCTCAGCCGTGCTCTCCCAGGAAGCCAGCGTGTGACCTTCCAGCCCGCGGACCGATGCTGCCGGCGGCCGCTCGCCCCCTGTGGGGGCCTTGCCTTGGGCTTCGGGCCGCTGCGTTCCGCCTTGCCACCTGGGCGTTTGGGGCTTTAGCTAGGAAAGAGGATGTCGCCTGCGGCTGCGAAGGCGCCAAAGCCCAGACCTCGCGCCTCCGCCCGCGACGGCGGCACTGGGGGCCCGGAGCGGGCGGGAGACGTGGATTACTGCGGGCTTCGGGGCCCTAGCCTCGACCGCGACGTTCGCTCGCATCGCGTACGGACAGAAACCCGTGATCTGAGAACCCTGGACTGGGACCAAGGAGCTAGGTAATTTCCGGCCCTAGTGATTACCAGCCTGAGCTTATGGCACCCACCAGTCCCCCAGTTGTCCTGCAGGTTTGAGTGGTATGGCTGCAAATAAACACGTGGCTTTCTAAGGCTCTTTGAAGAATTTAAAGTAACTTGCAAATCTTATCAATACGGACCCCTTGAGGTCTCACTCACCTTCACATCCAGGCCTCTTAGGGGAGCGCGGCTTTGGCTTTTCCAGGAAGGAATGGACTCCAACGGACAGGGAAAG

In [10]:
print('# base pairs: ', len(dataset['train'][0]['sequence']))

# base pairs:  2000


In [11]:
print('tokens: ', ' '.join(tokenizer.tokenize(dataset['train'][0]['sequence'])))

tokens:  GCGC TGAAG AAAG AACACC TGAA TCCGG AGCCCC AGCC CCCG TCGC TCG TTTGC GCG ACCCC AAAACC TGGCGGG ACG AGGG TCG CCCG ATGTC TAGCTGG AGGAG TTTGGGC TTGGC GCC TCCTTC GTG TAGTGGC TGAAGC TTCCC GGGTGC TCCTC ATTCC TGGGAAGGG AGTCGGG TGATGGG AACGG TGGGAGC TCAGCGC TCCG AGTCTGC GGCCGG GGGCC AGC GGGGC TGAAAC GCGCC TCCTG AGTGTTG AAAGC GGC AGGCGCC GGCC TTTGC ACCC AGAGGCC AGGGCGG ACGGGC AGG ATAAGC AGGC ACTGGG CCC AAGCC GCCC TTGGAGGC TCTGCC TGTTC TGATC TGAAGGC GGAGGC TCAGCC GTGC TCTCCC AGGAAGCC AGCG TGTG ACCTTCC AGCCC GCGG ACCG ATGCTGCC GGC GGCCGC TCGCC CCC TGTGGG GGCC TTGCC TTGGGC TTC GGGCC GC TGCG TTCCGCC TTGCCACC TGGGCG TTTGGGGC TTTAGC TAGG AAAGAGG ATGTC GCCTGC GGC TGCG AAGGC GCC AAAGCCC AGACCTC GCGCC TCCGCCC GCG ACGGC GGC ACTGGGGG CCCGG AGCGGGC GGGAGAC GTGG ATTACTGC GGGC TTC GGGGCCC TAGCC TCGACC GCG ACG TTCGC TCGC ATC GCG TACGG ACAGAA ACCC GTGATC TGAGAA CCC TGGAC TGGG ACCAAGG AGC TAGGTAA TTTCC GGCCC TAGTGATT ACCAGCC TGAGC TTATGGC ACCCACC AGTCCCCC AGTTG TCCTGC AGGTTTG AGTGG TATGGC TGC AAATAA ACACG 

In [12]:
print('# tokens: ', len(tokenizer.tokenize(dataset['train'][0]['sequence'])))

# tokens:  367


In [13]:
def preprocess_labels(example):
  example['label'] = example['promoter_presence']
  return example

dataset = dataset.map(preprocess_labels)

Map: 100%|████████████████████████████████████████████████████████████████| 1184/1184 [00:00<00:00, 25737.79 examples/s]


In [14]:
def preprocess_function(examples):
  # just truncate right, but for some tasks symmetric truncation from left and right is more reasonable
  # set max_length to 128 to make experiments faster
  return tokenizer(examples["sequence"], truncation=True, max_length=512) #max_length 128

In [15]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map: 100%|█████████████████████████████████████████████████████████████████| 1184/1184 [00:00<00:00, 7033.50 examples/s]


In [16]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [17]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sequence', 'promoter_presence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 10656
    })
    test: Dataset({
        features: ['sequence', 'promoter_presence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1184
    })
})

In [18]:
from transformers import TrainingArguments, Trainer
import numpy as np

#tokenizer.pad_token = tokenizer.eos_token 
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': (predictions==labels).sum() / len(labels)}

# change training hyperparameters to archive better quality
training_args = TrainingArguments(
    output_dir="ds_job_dna_1_run",
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_ratio=0.1,
    optim='adamw_torch',
    weight_decay=0.0,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    num_train_epochs=40,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/home/maris/miniconda3/envs/dnagpt/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.697300,0.587714,0.692568
2,0.508700,0.431638,0.793074
3,0.377900,0.399622,0.820101
4,0.278700,0.503666,0.818412
5,0.177200,0.577425,0.823480
6,0.109600,0.667173,0.833615
7,0.067400,1.082993,0.847128
8,0.050900,1.427295,0.820946
9,0.045200,1.225509,0.834459
10,0.032500,1.741163,0.827703


TrainOutput(global_step=21320, training_loss=0.07788993775509089, metrics={'train_runtime': 10790.8244, 'train_samples_per_second': 39.5, 'train_steps_per_second': 1.976, 'total_flos': 8.200294908670771e+16, 'train_loss': 0.07788993775509089, 'epoch': 40.0})